In [29]:
from math import pi
from cpu.systems.fan_geometry import ParametricBladeGeometry
from cosapp.systems import System

from pythonocc_helpers.create import CreateTopology, CreateRotation
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

class RotorGeometry(System):
    def setup(self):
        blade = self.add_child(
            ParametricBladeGeometry("blade"),
            pulling={
                "tip_radius": "tip_radius",
                "hub_to_tip_ratio": "hub_to_tip_ratio",
                "position": "blade_position",
                "dimension": "blade_dimension",
            },
        )

        self.add_inward("count", 2)
        self.add_inward("thickness", 1e-3, unit="m")
        self.add_outward("geometry", None)

    def compute(self):
        blades = []
        for i in range(self.count):
            trsf = BRepBuilderAPI_Transform(
                self.blade.geometry, CreateRotation.rotation_x(2 * pi / self.count * i)
            )
            blades.append(trsf.Shape())

        self.geometry = CreateTopology.make_compound(*blades)

row = RotorGeometry("blade")
blade = row.blade

In [42]:
from pythonocc_helpers.render import JupyterThreeJSRenderer

import ipywidgets as widgets


def grid(s, render):

    blade_slider = widgets.IntSlider(
        value=60,
        min=50,
        max=70,
        step=1,
        description='Blade count:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )

    def on_value_change(change):
        s.count = change['new']
        s.run_drivers()
        render.update_shape(s.geometry, uid="row");

    blade_slider.observe(on_value_change, names='value')

    return widgets.HBox([render.show(), widgets.VBox([blade_slider, ])])

render = JupyterThreeJSRenderer(view_size=(1800, 800), camera_target=(1., 0., 0.), camera_position=(-2., 0., 0.))

render_row = render.add_shape(row.geometry, uid="row", face_color="#156289", opacity=1., linear_deflection = 0.15, angular_deflection = 0.15, curvature_deflection=0.15)
grid(row, render)

In [48]:
blade.height_over_chord = 1.
blade.q_factor = 1.
blade.max_thickness_position = 0.3
blade.max_thickness_ratio = 0.05
blade.leading_tension = 0.3
blade.trailing_tension = 0.3
blade.inlet_angle = 60.
blade.exit_angle = 20.
blade.stacking_parameter = 0.
blade.stacking_angle = 0.
blade.swirl = 0.

row.tip_radius = 0.8
row.hub_to_tip_ratio = 0.85
row.count = 60

row.run_drivers()

render.update_shape(row.geometry, uid="row");